In [ ]:
import pandas as pd
import duckdb

In [ ]:
bikes_table = duckdb.read_csv("https://cycling.data.tfl.gov.uk/ActiveTravelCountsProgramme/2024%20W2%20autumn-Cycleways.csv")

In [ ]:
duckdb.sql("select * from bikes_table where Weather!='Dry'")

In [ ]:
bikes_table.to_parquet('cycleways.parquet')

In [ ]:
#schema has changes somewhere so load only as of 2020
duckdb.read_csv("s3://cycling.data.tfl.gov.uk/usage-stats/*JourneyDataExtract*2024.csv").to_parquet("2024_journeys.parquet")

In [ ]:
duckdb.sql("s3://cycling.data.tfl.gov.uk/usage-stats/*2025.csv").to_parquet("2025_journeys.parquet")

In [ ]:
"""select number, """

In [ ]:
duckdb.read_parquet("2025_journeys.parquet").show()

In [ ]:
duckdb.read_json("https://cycling.data.tfl.gov.uk/cycling-load.json").df()

In [ ]:
! wget https://cycling.data.tfl.gov.uk/cycling-load.json

In [ ]:
import json 

with open("cycling-load.json") as f:
    json_data = json.load(f)

In [ ]:
[(x["url"].split("/")[-1]).split(".csv")[0] for x in json_data["entries"]]

In [ ]:
import duckdb
import re


# Connect to DuckDB
con = duckdb.connect(database="bike_rentals")


# Loop through tables and create them in DuckDB
for url in [x["url"] for x in json_data["entries"] if x["url"].endswith(".csv")]:
    table_suffix = re.sub("[^A-Za-z0-9 ]+", "", url.split("Journey")[0])
    table_name= f"table_{table_suffix}"
    # Create a table from csv files
    con.execute(f"""
        CREATE TABLE {table_name} AS 
        SELECT * FROM read_csv('{url}');
    """)

# Verify the created tables
print(con.execute("SHOW TABLES").fetchall())

In [ ]:
con.execute("SHOW TABLES").df()